In [1]:
import os
import numpy as np
import imageio
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
from utils import *

In [2]:
tMax = 20
stock = 340
strikeMin = stock*0.5
strikeMax = stock*1.5

###
n = int((strikeMax - strikeMin)/0.01 + 1)
strike = np.linspace(strikeMin, strikeMax, n)

### t days until expiration
valueT = np.zeros((tMax+1, n))
deltaT = np.zeros((tMax+1, n))
deltaFactorT = np.zeros((tMax+1, n))
thetaT = np.zeros((tMax+1, n))
thetaFactorT = np.zeros((tMax+1, n))
for t in range(tMax+1)[::-1]:
    value = BlackScholes(strike, stock, t)
    valueT[t] = value
    theta = Greeks(strike, stock, t, greek="theta")
    delta = Greeks(strike, stock, t, greek="delta")
    thetaT[t] = theta
    deltaT[t] = delta
    thetaFactorT[t] = np.where(value > 1e-50, theta/value, np.nan)
    deltaFactorT[t] = np.where(value > 1e-50, stock*delta/value, np.nan)

### Normalizing Data

# normalizing stock value
idx = np.argmin(np.abs(stock - strike))
moneyness = (strike - stock) / strike
scale = 50

In [3]:
# Generating images
files = []
#norm = plt.Normalize(vmin = np.min(moneyness), vmax = np.max(moneyness))
norm = plt.Normalize(vmin = -1, vmax = 1)

for t in range(1, tMax+1)[::-1]:
    fig, ax = plt.subplots(figsize = [11, 8], tight_layout = True)

    title = r"$\bf{Leverage\ versus\ Daily\ Theta\ Loss\ of\ Call\ Option\ Contracts}$"
    title += "\n"
    title += r"$\rm{Days\ to\ Expiration:\ }$"
    title += "%2s" % t
    plt.title(title, family = "monospace", fontsize = 16)
    
    x, y = thetaFactorT[t], deltaFactorT[t]
    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    lc = LineCollection(segments, cmap='viridis', norm=norm)
    lc.set_array(np.tanh(scale*moneyness))
    lc.set_linewidth(6)
    line = ax.add_collection(lc)
    cbar = fig.colorbar(line)
    cbar.set_ticks([-1, 0, 1])
    cbar.set_ticklabels(["ITM", "ATM", "OTM"])
    cbar.ax.tick_params(labelsize=14) 
    
    plt.scatter([x[idx]], [y[idx]], c = [[0.127568, 0.566949, 0.550556, 1.0]], marker='o', s = 100)

    plt.xlim([0, -0.5])
    plt.gca().set_xticklabels(['{:.0f}%'.format(x*100) for x in plt.gca().get_xticks()]);
    plt.xticks(fontsize = 14)
    plt.ylim([0, 140])
    plt.yticks(fontsize = 14)
    plt.grid()
    
    xlabel = r"$\bf{Percentage\ Theta\ Loss}$"
    xlabel += "\n"
    xlabel += r"$\rm{\left(\frac{Theta}{option\ price}\right)}$"
    plt.xlabel(xlabel, fontsize = 16)
    
    ylabel = r"$\bf{Leverage}$"
    ylabel += "\n"
    ylabel += r"$\rm{\left(Delta\times\frac{stock\ price}{option\ price}\right)}$"
    plt.ylabel(ylabel, fontsize = 16)
    
    fname = "DeltaVersusTheta_%02d.png" % t
    files.append(fname)
    plt.savefig(fname)
    plt.close()

In [4]:
### Making movie animation
images = []
for i in range(len(files)):
    fname = files[i]
    # make first and last frames longer
    if i == 0 or i == len(files)-1:
        for _ in range(2):
            images.append(imageio.imread(fname))
    #
    images.append(imageio.imread(fname))
imageio.mimsave('DeltaVersusTheta.gif', images, duration=0.5)

### cleanup
for fname in files:
    os.remove(fname)